# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
import zipfile
import csv
from nltk.corpus import stopwords
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.probability import ConditionalFreqDist

In [2]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    string = re.sub(r'http\S+', '', s)
    return re.sub('[^A-Za-z]+', ' ', string).lower().strip()

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)

def stem_and_lemmatize(l):
    
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    l2 = []
    
    for w in l:
        s = ps.stem(w)
        s = lemmatizer.lemmatize(s)
        l2 += [s]
    
    return l2


def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words('english')

    return [w for w in l if w not in stop_words]

In [3]:
import pandas as pd
df = pd.read_csv('C:\\Users\\sanch\\OneDrive\\Documentos\\Dataset\\sentimental140b.csv', 
                 names=['target', 'id', 'date', 'flag', 'user', 'text'], 
                 encoding='latin-1')
sample = df.sample(20000)
sample['target'] = sample['target'].replace(4, 1)

In [4]:
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
sample['text_processed'] = sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
sample

,target,id,date,flag,user,text,text_processed
795754,0,2327482047,Thu Jun 25 08:24:11 PDT 2009,NO_QUERY,loud_mouth_ann,I STILL NEED A JOB,"[still, need, job]"
1374288,1,2051418499,Fri Jun 05 21:58:35 PDT 2009,NO_QUERY,Mat2DaW,guess so ?,[guess]
321846,0,2003949346,Tue Jun 02 07:25:11 PDT 2009,NO_QUERY,qUibAbii,Finished my apes exam and it was really hard,"[finish, ape, exam, wa, realli, hard]"
192176,0,1969792927,Sat May 30 01:14:53 PDT 2009,NO_QUERY,Misosekushii,can someone buy me adrink? Faaa real!!!!!,"[someon, buy, adrink, faaa, real]"
166591,0,1961366316,Fri May 29 09:23:34 PDT 2009,NO_QUERY,Zoraida1980,Headache,[headach]
...,...,...,...,...,...,...,...
1174645,1,1980985511,Sun May 31 08:08:58 PDT 2009,NO_QUERY,Heidi_vb,@madmohican spendin it in bed wiv my lover hav...,"[madmohican, spendin, bed, wiv, lover, havin, ..."
940673,1,1794012851,Thu May 14 04:42:11 PDT 2009,NO_QUERY,CecyTxGaL,I made it to school. Traffic was pretty good t...,"[made, school, traffic, wa, pretti, good, toda..."
162348,0,1957523936,Fri May 29 00:46:19 PDT 2009,NO_QUERY,Mnemosyne,I've got a dentist's appointment soon! The dr...,"[got, dentist, appoint, soon, drill, feel, lik..."
474523,0,2177357500,Mon Jun 15 05:57:52 PDT 2009,NO_QUERY,CourtneyJurek,I woke up at 830 on my only day off,"[woke, onli, day]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [6]:
cfdist = nltk.FreqDist()

for tweet in sample['text_processed']:
    for word in tweet:
        cfdist[word] += 1

all_word = list(cfdist.keys())[:5000]
all_word

['still',
 'need',
 'job',
 'guess',
 'finish',
 'ape',
 'exam',
 'wa',
 'realli',
 'hard',
 'someon',
 'buy',
 'adrink',
 'faaa',
 'real',
 'headach',
 'flawless',
 'much',
 'appreci',
 'reciev',
 'gift',
 'love',
 'sure',
 'slack',
 'tweet',
 'thi',
 'color',
 'remind',
 'johnni',
 'quest',
 'cartoon',
 'rerurn',
 'one',
 'traceylien',
 'shame',
 'yeah',
 'pasta',
 'pretti',
 'finicki',
 'easiest',
 'delici',
 'make',
 'though',
 'missmath',
 'cabachand',
 'allez',
 'vou',
 'twitter',
 'aussi',
 'comm',
 'tic',
 'cegep',
 'al',
 'de',
 'aqpc',
 'get',
 'japanes',
 'food',
 'heart',
 'teriaki',
 'miss',
 'dirti',
 'asian',
 'ken',
 'matt',
 'enjoy',
 'ur',
 'random',
 'happi',
 'face',
 'lol',
 'hope',
 'good',
 'u',
 'keep',
 'touch',
 'jen',
 'last',
 'night',
 'stay',
 'start',
 'monday',
 'may',
 'mad',
 'lt',
 'seqalab',
 'believ',
 'sweetwat',
 'gone',
 'condol',
 'mikegarrick',
 'goodnight',
 'mike',
 'beach',
 'tomorrow',
 'derricklbrigg',
 'ya',
 'like',
 'nice',
 'thing',
 '

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [8]:
def find_features(document):
    words = set(document)
    features = {}
    for w in all_word:
        features[w] = (w in words)
    
    return features
        
features = [(find_features(tweet), target) for (tweet, target) in list(zip(sample['text_processed'], sample['target']))]
print(len(features))

20000


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [10]:
train_set, test_set = features[:10000], features[10000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [12]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.7231
Most Informative Features
                 headach = True                0 : 1      =     18.3 : 1.0
                     www = True                1 : 0      =     13.6 : 1.0
                   shame = True                0 : 1      =     12.8 : 1.0
                 stomach = True                0 : 1      =     12.8 : 1.0
                   stuck = True                0 : 1      =     12.4 : 1.0
                 depress = True                0 : 1      =     11.5 : 1.0
                    sick = True                0 : 1      =     11.1 : 1.0
                    poor = True                0 : 1      =     10.8 : 1.0
                    ruin = True                0 : 1      =      9.5 : 1.0
                     sad = True                0 : 1      =      9.2 : 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here